# Introducción

Este notebook, tiene como objetivo principal realizar el análisis de la relación entre el nivel educativo y la tasa de desempleo en la Unión Europea para jóvenes de 15 a 29 años. Utilizamos dos bases de datos extraídas de Eurostat:

1. **Youth unemployment by sex, age and educational attainment level**
2. **Young people by educational attainment level, sex and age**

El propósito de este análisis es comprobar nuestra hipótesis de que existe una relación inversa entre el nivel educativo y el desempleo. Además, queremos evaluar si esta relación varía por género y en diferentes países de Europa.

### Pasos Realizados en el Código

**Preprocesamiento de Datos:**
   - **Carga de Datos:** Importamos las bases de datos desde archivos CSV.
   - **Limpieza y Estandarización:** Eliminamos columnas innecesarias, renombramos y estandarizamos los datos para asegurar la consistencia entre las dos bases de datos.
   - **Fusión de Datos:** Combinamos las dos bases de datos en una única base de datos usando las columnas comunes: sexo, edad, país y nivel educativo.

**Transformación de Datos:**
   - **Creación de Variables:** Calculamos una nueva columna de nivel educativo ponderado, ya que la tasa proporcionada en los datos originales no representaba el nivel educativo real.

Este código está diseñado para proporcionar una comprensión clara de cómo el nivel educativo afecta la tasa de desempleo y para ayudar a la creación de una empresa educativa ficticia al identificar oportunidades basadas en datos reales.


### 1. Importar Librerías, Funciones y Bases de Datos

Primero, importamos las librerías necesarias y el archivo `.py` que contiene todas las funciones de limpieza de datos.

In [1]:
import pandas as pd
import numpy as np

from functions import *

Después, importamos las bases de datos extraídas de Eurostat

In [2]:
url = "data/edat_lfs_9903__custom_12827842_linear.csv"
education = read_data(url)

In [ ]:
url2 = "data/yth_empl_090__custom_12832588_linear.csv"
unemployment = read_data(url2)

### 2. Ejecutar el Proceso de Limpieza

A continuación, ejecutamos el proceso de limpieza completo utilizando las diferentes funciones que hemos definido en el script `functions`.

Además, mostramos las cinco primeras filas de cada base de datos para demostrar que la limpieza ha sido efectiva.

In [3]:
pd.set_option('display.max_columns', None)
education = clean_columns(education)
column_names_education(education)

education = change_value(education) 
education = change_name_country_education(education) 
education = nulos(education)

education = drop_rows(education)

education.head()

,sex,level_education,age,country,year,value_education
0,Females,ESO,15-29,Austria,2014,27.9
1,Females,ESO,15-29,Austria,2015,27.5
2,Females,ESO,15-29,Austria,2016,27.3
3,Females,ESO,15-29,Austria,2017,27.2
4,Females,ESO,15-29,Austria,2018,27.1


In [5]:
unemployment = clean_columns(unemployment)
column_names_unemployment(unemployment)

unemployment = change_value(unemployment)
unemployment = change_name_country_unemployment(unemployment)
unemployment = nulos(unemployment)

unemployment = drop_rows(unemployment)
unemployment.head()

,sex,age,level_education,country,year,value_unemployment
0,Females,15-29,ESO,Austria,2014,15.7
1,Females,15-29,ESO,Austria,2015,15.3
2,Females,15-29,ESO,Austria,2016,17.1
3,Females,15-29,ESO,Austria,2017,16.8
4,Females,15-29,ESO,Austria,2018,13.4


### 3. Procesamiento de duplicados

Encontramos que el dataset `unemployment` tiene la mayoría de filas duplicadas, por lo que procedemos a eliminarlas.

Primero, mostramos el número de filas duplicadas y después las eliminamos.

In [6]:
unemployment.duplicated(subset=["sex", "level_education", "age", "country", "year"]).sum()

4416

In [7]:
unemployment = drop_duplicates(unemployment)

### 4. Combinación de bases de datos

Una vez tenemos estandarizadas ambas columnas, las combinamos utilizando las columnas comunes `sex`, `age`, `level_education`, `country` y `year`.

In [8]:
df_limpio = comb_merg(unemployment, education)
df_limpio

,sex,age,level_education,country,year,value_unemployment,value_education
0,Females,15-29,ESO,Austria,2014,15.7,27.9
1,Females,15-29,ESO,Austria,2015,15.3,27.5
2,Females,15-29,ESO,Austria,2016,17.1,27.3
3,Females,15-29,ESO,Austria,2017,16.8,27.2
4,Females,15-29,ESO,Austria,2018,13.4,27.1
...,...,...,...,...,...,...,...
4666,Total,15-29,UNIVERSIDAD,United Kingdom,2015,5.4,28.9
4667,Total,15-29,UNIVERSIDAD,United Kingdom,2016,5.0,29.3
4668,Total,15-29,UNIVERSIDAD,United Kingdom,2017,4.6,29.5
4669,Total,15-29,UNIVERSIDAD,United Kingdom,2018,4.4,30.4


### 5. Procesamiento de valores nulos

Al realizar la combinación, varios valores de la columna `value_unemployment`se convirtieron en NaN.

Para resolver esto, eliminamos aquellos países que no contaban con los datos suficientes o válidos y también utilizamos `fill_nan()`. 

Esta función rellena los valores NaN con el elemento de la fila anterior. 

In [9]:
df_limpio = fill_nan(df_limpio)
df_limpio

,sex,age,level_education,country,year,value_unemployment,value_education
0,Females,15-29,ESO,Austria,2014,15.7,27.9
1,Females,15-29,ESO,Austria,2015,15.3,27.5
2,Females,15-29,ESO,Austria,2016,17.1,27.3
3,Females,15-29,ESO,Austria,2017,16.8,27.2
4,Females,15-29,ESO,Austria,2018,13.4,27.1
...,...,...,...,...,...,...,...
4666,Total,15-29,UNIVERSIDAD,United Kingdom,2015,5.4,28.9
4667,Total,15-29,UNIVERSIDAD,United Kingdom,2016,5.0,29.3
4668,Total,15-29,UNIVERSIDAD,United Kingdom,2017,4.6,29.5
4669,Total,15-29,UNIVERSIDAD,United Kingdom,2018,4.4,30.4


Demostramos que tras el procesamiento de nulos, no queda ninguno.

In [10]:
df_limpio.isnull().sum()

sex                   0
age                   0
level_education       0
country               0
year                  0
value_unemployment    0
value_education       0
dtype: int64

### 6. Columna Índice educativo

La columna `value_education` no muestra el nivel educativo real, por lo que utilizando esa columna junto con la columna `level_education` y `valor_educacion` calculamos el `indice_educativo`.

`valor_educacion` es una columna que representa el nivel de educación del 1 al 5 basándose en `level_education`. 
Siendo:
1. ESO == 1
2. FP == 2
3. BACH-FP == 3
4. BACH == 4
5. UNIVERSIDAD == 5

Con esto, calculamos la media ponderada para `indice_educativo`con `valor_educacion`y `value_education`.

Aquí encontramos un nuevo problema, **las filas donde `sex`== `Total` tenían resultados nulos**. Lo resolvemos utilizando la media obtenida en ambos géneros para cada país. 

In [11]:
df_resultado = calcular_indice_educativo(df_limpio)
df_resultado

c:\Users\Bapa\Documents\GitHub\project-1\project-ue\functions.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['ponderado'] = df_filtrado['value_education'] * df_filtrado['valor_educacion']


,sex,age,level_education,country,year,value_unemployment,value_education,valor_educacion,indice_educativo
0,Females,15-29,ESO,Austria,2014,15.7,27.9,1,2.84
1,Females,15-29,ESO,Austria,2015,15.3,27.5,1,2.89
2,Females,15-29,ESO,Austria,2016,17.1,27.3,1,2.91
3,Females,15-29,ESO,Austria,2017,16.8,27.2,1,2.92
4,Females,15-29,ESO,Austria,2018,13.4,27.1,1,2.92
...,...,...,...,...,...,...,...,...,...
4666,Total,15-29,UNIVERSIDAD,United Kingdom,2015,5.4,28.9,5,3.21
4667,Total,15-29,UNIVERSIDAD,United Kingdom,2016,5.0,29.3,5,3.21
4668,Total,15-29,UNIVERSIDAD,United Kingdom,2017,4.6,29.5,5,3.21
4669,Total,15-29,UNIVERSIDAD,United Kingdom,2018,4.4,30.4,5,3.21


### 7. Creación de csv
Por último, creamos un archivo `.csv`con el dataframe limpio.


In [13]:
create_csv(df_resultado)